In [ ]:
# Installer les packages nécessaires (plotly, ipywidgets)
# Cette cellule peut être cachée dans la version finale avec :tags: [remove-input]
%pip install plotly --quiet

## 🌟 Atelier interactif

Découvrez plusieurs ateliers interactifs conçus pour mieux comprendre la théorie de Lane et Taylor :

1. 🧮 **Feuillet de calcul de la teneur de coupure optimale**  
   Visualisez la teneur de coupure optimale, identifiez sa valeur et son type.

2. 📊 **Calcul des réserves selon une loi lognormale**  
   Explorez le calcul des variables \(x_c\) et \(g_c\) en fonction de la teneur de coupure, dans le cadre d’une loi lognormale.

3. 🔍 **Analyse de sensibilité de la formule de Lane et Taylor**  
   Tous les paramètres sont fixés sauf un, qui varie ; plusieurs cas sont présentés pour analyser leur influence.



### 🧮 1 - Feuillet de calcul — Teneur de coupure optimale

Ce module interactif vous permet d’explorer la théorie de **Lane et Taylor** pour déterminer les teneurs de coupure :

- **Teneurs limites**  
- **Teneurs d’équilibres**  
- **Teneur optimale**

### 🛠️ Instructions

1. Activez l’activité ci-dessous.  
2. Des champs apparaîtront pour saisir les **paramètres**.  
3. Le **graphique dynamique** s’affichera automatiquement, illustrant les trois courbes de profits.

💡 *Ce calculateur est un excellent outil pour valider vos travaux pratiques.*

In [53]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import matplotlib.patches as patches
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m, y, p, k, h, f, F, M, H, K, moy, s2, x_expand=1, y_expand=5):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre comme dans ton code initial
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Limites des axes : X = c_opt ± x_expand, Y = profit_opt ± y_expand
    xmin, xmax = c_opt - x_expand, c_opt + x_expand
    ymin, ymax = profit_opt - y_expand, profit_opt + y_expand

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    df = pd.DataFrame([
    ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
    ["Limite C1",f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
    ["Limite C2",f"{c2:.2f} %",          f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
    ["Limite C3",f"{c3:.2f} %",         f"{v3[np.argmax(v3)]:.2f} $", "Marché"],
    ["Équilibre C1-C2", f"{ceq[0]:.2f} %",       f"{v1[np.argmin(np.abs(v1 - v2))]:.2f} $", "Mine–Conc."],
    ["Équilibre C2-C3", f"{ceq[1]:.2f} %",       f"{v2[np.argmin(np.abs(v2 - v3))]:.2f} $", "Conc.–Marché"],
    ["Équilibre C1-C3", f"{ceq[2]:.2f} %",       f"{v1[np.argmin(np.abs(v1 - v3))]:.2f} $", "Mine–Marché"]
    ], columns=["Type de coupure", "Teneur", "Profit associé", "Remarque"])

    # Affichage du tableau
    print(df.to_string(index=False))

# Widgets FloatText pour paramètres, sans xmin, xmax, ymin, ymax
style = {'description_width': '170px'}
layout = widgets.Layout(width='280px')

m_w = widgets.FloatText(value=1.32, description='Coût minage (m)', style=style, layout=layout)
y_w = widgets.FloatText(value=0.87, description='Taux récupération (y)', style=style, layout=layout)
p_w = widgets.FloatText(value=600, description='Prix métal (p)', style=style, layout=layout)
k_w = widgets.FloatText(value=0, description='Coût fonderie (k)', style=style, layout=layout)
h_w = widgets.FloatText(value=3.41, description='Coût traitement (h)', style=style, layout=layout)
f_w = widgets.FloatText(value=11.9, description='Frais fixes (f)', style=style, layout=layout)
F_w = widgets.FloatText(value=0, description='Coût opportunité (F)', style=style, layout=layout)
M_w = widgets.FloatText(value=12, description='Capacité minage (M)', style=style, layout=layout)
H_w = widgets.FloatText(value=3.9, description='Capacité traitement (H)', style=style, layout=layout)
K_w = widgets.FloatText(value=0.085, description='Capacité marché (K)', style=style, layout=layout)
moy_w = widgets.FloatText(value=1, description='Moyenne lognormale (%)', style=style, layout=layout)
s2_w = widgets.FloatText(value=4, description='Variance lognormale (%)^2', style=style, layout=layout)

# Widgets pour élargissement de la zone en x et y
x_expand_w = widgets.FloatText(value=1, description='Élargissement X', style=style, layout=layout)
y_expand_w = widgets.FloatText(value=1, description='Élargissement Y', style=style, layout=layout)

ui = widgets.VBox([
    widgets.HBox([m_w, y_w, p_w, k_w]),
    widgets.HBox([h_w, f_w, F_w, M_w]),
    widgets.HBox([H_w, K_w, moy_w, s2_w]),
    widgets.HBox([x_expand_w, y_expand_w])
])

out = widgets.Output()

def update_plot(change):
    with out:
        out.clear_output(wait=True)
        lane(
            m=m_w.value, y=y_w.value, p=p_w.value, k=k_w.value, h=h_w.value,
            f=f_w.value, F=F_w.value, M=M_w.value, H=H_w.value, K=K_w.value,
            moy=moy_w.value, s2=s2_w.value,
            x_expand=x_expand_w.value, y_expand=y_expand_w.value
        )

for w in [m_w, y_w, p_w, k_w, h_w, f_w, F_w, M_w, H_w, K_w, moy_w, s2_w, x_expand_w, y_expand_w]:
    w.observe(update_plot, names='value')

display(ui, out)

update_plot(None)

Output()

### 📊 Calcul des réserves selon une loi lognormale

**Atelier interactif pour comprendre comment les paramètres \($x_c$\) et \($g_c$\) sont calculés**

1. La valeur de $x_c$ correspond à la proportion, représentée par la zone hachurée dans le graphique de la fonction de densité ($x_c = P(X > c)$) ; elle s'obtient aussi comme $x_c = 1 - P(X < c)$ à partir de la fonction de répartition.

2. La valeur de \($g_c$\) correspond à la moyenne des teneurs situées dans cette zone hachurée (c’est-à-dire pour $g_c = E[X|X>c]$ ) sur le graphique de la fonction de densité.

> **Note :** Les paramètres \($x_c$\) et \($g_c$\) dépendent de la distribution statistique des teneurs dans le gisement.  
> Ici, on suppose une loi lognormale de moyenne $moy$ et de variance $s^2$.  
> Toutefois, toute autre distribution pourrait être utilisée selon les caractéristiques réelles du gisement.  
> Cela souligne l’importance de disposer d’une quantité suffisante d’informations pour bien caractériser la distribution des teneurs.


In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm
import ipywidgets as widgets
from IPython.display import display

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def plot_interactive(moy, s2, c_cut):
    c_range = np.linspace(0.001, 8, 500)
    tc_vals, qc_vals, mc_vals = reserve(moy, s2, c_range, itype=1)
    tc_cut, qc_cut, mc_cut = reserve(moy, s2, c_cut, itype=1)

    sigma = np.sqrt(np.log(s2 / moy**2 + 1))
    mu = np.log(moy) - sigma**2 / 2
    x = np.linspace(0.001, 8, 500)
    cdf = lognorm.cdf(x, s=sigma, scale=np.exp(mu))
    pdf = lognorm.pdf(x, s=sigma, scale=np.exp(mu))
    cdf_cut = lognorm.cdf(c_cut, s=sigma, scale=np.exp(mu))
    pdf_cut = lognorm.pdf(c_cut, s=sigma, scale=np.exp(mu))


    fig, axs = plt.subplots(2, 2, figsize=(14,10))
    axs = axs.flatten()  # Pour accès simple 0 à 3

    # 1. x_c
    axs[0].plot(c_range, tc_vals, label=r'$x_c = P(X > c)$')
    axs[0].axvline(c_cut, color='red', linestyle='--', label=f'c = {c_cut:.2f}')
    axs[0].scatter(c_cut, tc_cut, color='red')
    axs[0].set_xlim(0, 8)
    axs[0].set_ylim(0, 1)
    axs[0].set_xlabel('Teneur de coupure ($c$)')
    axs[0].set_ylabel('Proportion au-dessus ($x_c$)')
    axs[0].set_title(r'$x_c$ en fonction de $c$')
    axs[0].legend(loc='upper right')
    axs[0].grid()

    # 2. g_c
    axs[1].plot(c_range, mc_vals, label=r'$g_c = E[X|X>c]$')
    axs[1].axvline(c_cut, color='red', linestyle='--', label=f'c = {c_cut:.2f}')
    axs[1].scatter(c_cut, mc_cut, color='red')
    axs[1].set_xlim(0, 8)
    axs[1].set_ylim(0, 15)
    axs[1].set_xlabel('Teneur de coupure ($c$)')
    axs[1].set_ylabel('Teneur moyenne ($g_c$)')
    axs[1].set_title(r'$g_c$ en fonction de $c$')
    axs[1].legend(loc='upper right')
    axs[1].grid()

    # 3. CDF
    axs[2].plot(x, cdf, label='Fonction de répartition')
    axs[2].fill_between(x, 0, cdf, where=(x > c_cut), color='grey', alpha=0.3, hatch='///', label=r'$x > c$')
    axs[2].axvline(c_cut, color='red', linestyle='--', label=f'c = {c_cut:.2f}')
    axs[2].scatter(c_cut, cdf_cut, color='red')
    axs[2].set_xlim(0, 8)
    axs[2].set_ylim(0, 1)
    axs[2].set_xlabel('Teneur de coupure ($c$)')
    axs[2].set_ylabel('$F(c) = P(X < c)$')
    axs[2].set_title('Fonction de répartition lognormale')
    axs[2].legend(loc='upper right')
    axs[2].grid()

    # 4. PDF
    axs[3].plot(x, pdf, label='Fonction de densité')
    axs[3].fill_between(x, 0, pdf, where=(x > c_cut), color='grey', alpha=0.3, hatch='///', label=r'$x > c$')
    axs[3].axvline(c_cut, color='red', linestyle='--', label=f'c = {c_cut:.2f}')
    axs[3].scatter(c_cut, pdf_cut, color='red')
    axs[3].set_xlim(0, 8)
    axs[3].set_ylim(0, max(pdf)+0.1)
    axs[3].set_xlabel('Teneur de coupure ($c$)')
    axs[3].set_ylabel('Densité')
    axs[3].set_title('Fonction de densité lognormale')
    axs[3].legend(loc='upper right')
    axs[3].grid()

    plt.tight_layout()
    plt.show()


# Widgets
w_moy = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.05, description='Moyenne')
w_s2 = widgets.FloatSlider(value=4, min=1, max=8, step=0.25, description='Variance')
w_c = widgets.FloatSlider(value=0.5, min=0.001, max=8, step=0.2, description='Teneur coupure')

ui = widgets.VBox([w_moy, w_s2, w_c])
out = widgets.interactive_output(plot_interactive, {'moy': w_moy, 's2': w_s2, 'c_cut': w_c})

display(ui, out)



Output()

## 🔍 Analyse de sensibilité

**Ici, nous allons faire varier un paramètre à la fois et observer son impact sur la teneur de coupure optimale et les profits**


#### Impact des coûts variables de minage (par tonne de matériau minéralisé) – $m$

Faites varier les coûts variables de minage ($m$) et observez son impact sur la teneur de coupure optimale ($c_{opt}$) et le profit net généré par une unité de matériau minéralisé ($v_{opt}$).

In [59]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.25

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widget interactif — uniquement m est variable, les autres fixés
w_m = widgets.BoundedFloatText(value=1.32, min=0.5, max=2.5, step=0.05, description='m', layout=layout, style=style)


def update(m):
    lane(m=m)

widgets.interact(update, m=w_m)


interactive(children=(BoundedFloatText(value=1.32, description='m', layout=Layout(width='300px'), max=2.5, min…

<function __main__.update(m)>

#### Impact des coûts variables de traitement (par tonne de minerai) – $h$

Faites varier les coûts variables de traitement par tonne de minerai ($h$) et observez leur impact sur la teneur de coupure optimale ($c_{\text{opt}}$) ainsi que sur le profit net par tonne de matériau minéralisé traité ($v_{\text{opt}}$).

Faite de meme avec le coût de mise en marché d’une tonne de métal (fonderie, transport, etc.) ($k$) et le prix de vente d’une tonne de métal ($p$).

> 💡 **Note** : Le coût de mise en marché d’une tonne de métal (fonderie, transport, etc.) ($k$), lorsqu’il est pris en compte, agit de manière similaire aux coûts variables de traitement ($h$) dans le modèle.

> 💡 **Note** : Le prix de vente d’une tonne de métal ($p$), lorsqu’il est pris en compte, a un effet inverse à celui des coûts variables de traitement ($h$) dans le modèle.


In [68]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.6

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_h = widgets.BoundedFloatText(value=3.41, min=1, max=4.5, step=0.1, description='h', layout=layout, style=style)
w_k = widgets.BoundedFloatText(value=0, min=0, max=200, step=10, description='k', layout=layout, style=style)
w_p = widgets.BoundedFloatText(value=600, min=400, max=800, step=10, description='p', layout=layout, style=style)

def update(h, k, p):
    lane(h=h, k=k, p=p)

widgets.interact(update, h=w_h, k=w_k, p=w_p)

interactive(children=(BoundedFloatText(value=3.41, description='h', layout=Layout(width='300px'), max=4.5, min…

<function __main__.update(h, k, p)>

#### Impact des frais fixes (administration, ingénierie, capital) – $f$

Faites varier les frais fixes ($f$), qui incluent des coûts comme l’administration, l’ingénierie et le capital, et observez leur impact sur la teneur de coupure optimale ($c_{\text{opt}}$) ainsi que sur le profit net par tonne de matériau minéralisé traité ($v_{\text{opt}}$).

> 💡 **Note** : Le coût d’opportunité ($F$), lorsqu’il est pris en compte, agit de la même manière que les frais fixes dans le modèle.


In [72]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.75

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_f = widgets.BoundedFloatText(value=11.9, min=2, max=16, step=0.25, description='f', layout=layout, style=style)
w_F = widgets.BoundedFloatText(value=0, min=0, max=8, step=0.5, description='F', layout=layout, style=style)

def update(f, F):
    lane(f=f, F=F,)

widgets.interact(update, f=w_f, F=w_F)

interactive(children=(BoundedFloatText(value=11.9, description='f', layout=Layout(width='300px'), min=2.0, ste…

<function __main__.update(f, F)>

#### Impact de la capacité de minage (matériau minéralisé) – $M$

Faites varier la capacité de minage (matériau minéralisé) ($M$) et observez son impact sur la teneur de coupure optimale ($c_{opt}$) et le profit net généré par une unité de matériau minéralisé ($v_{opt}$).

In [73]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.75

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_M = widgets.BoundedFloatText(value=12, min=2, max=16, step=0.25, description='M', layout=layout, style=style)

def update(M):
    lane(M=M,)

widgets.interact(update, M=w_M)

interactive(children=(BoundedFloatText(value=12.0, description='M', layout=Layout(width='300px'), max=16.0, mi…

<function __main__.update(M)>

#### Impact de la capacité de traitement (minerai sélectionné) – $H$

Faites varier la capacité de traitement (minerai sélectionné) ($H$) et observez son impact sur la teneur de coupure optimale ($c_{opt}$) et le profit net généré par une unité de matériau minéralisé ($v_{opt}$).

In [74]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.75

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_H = widgets.BoundedFloatText(value=3.9, min=2, max=6, step=0.1, description='H', layout=layout, style=style)

def update(H):
    lane(H=H,)

widgets.interact(update, H=w_H)

interactive(children=(BoundedFloatText(value=3.9, description='H', layout=Layout(width='300px'), max=6.0, min=…

<function __main__.update(H)>

#### Impact de la capacité du marché (métal) – $K$

Faites varier la capacité du marché (métal) ($K$) et observez son impact sur la teneur de coupure optimale ($c_{opt}$) et le profit net généré par une unité de matériau minéralisé ($v_{opt}$).

In [76]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.75

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_K = widgets.BoundedFloatText(value=0.085, min=0.01, max=0.16, step=0.002, description='K', layout=layout, style=style)

def update(K):
    lane(K=K,)

widgets.interact(update, K=w_K)

interactive(children=(BoundedFloatText(value=0.085, description='K', layout=Layout(width='300px'), max=0.16, m…

<function __main__.update(K)>

#### Impact de la moyenne du gisement

Faites varier la moyennne des teneurs du gisement et observez son impact sur la teneur de coupure optimale ($c_{opt}$) et le profit net généré par une unité de matériau minéralisé ($v_{opt}$).

In [80]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.75

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_moy = widgets.BoundedFloatText(value=1, min=0.5, max=1.5, step=0.05, description='moy', layout=layout, style=style)

def update(moy):
    lane(moy=moy,)

widgets.interact(update, moy=w_moy)

interactive(children=(BoundedFloatText(value=1.0, description='moy', layout=Layout(width='300px'), max=1.5, mi…

<function __main__.update(moy)>

#### Impact de la variance du gisement

Faites varier la variance des teneurs du gisement et observez son impact sur la teneur de coupure optimale ($c_{opt}$) et le profit net généré par une unité de matériau minéralisé ($v_{opt}$).

In [78]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def reserve(moy, s2, c, itype=1):
    c = np.array(c)
    m = moy
    if itype == 1:
        b = np.sqrt(np.log(s2 / m**2 + 1))
        u = np.log(m) - 0.5 * b**2
        n1 = (np.log(m / c) / b) - b / 2
        n2 = n1 + b
        tc = norm.cdf(n1)
        qc = m * norm.cdf(n2)
        mc = qc / tc
    else:
        s = np.sqrt(s2)
        tc = norm.cdf((m - c) / s)
        qc = m * tc + s * norm.pdf((c - m) / s)
        mc = qc / tc
    return tc, qc, mc

def lane(m=1.32, y=0.87, p=600, k=0, h=3.41, f=11.9, F=0, M=12, H=3.9, K=0.085, moy=1, s2=4):
    cmax = (moy + 0.5 * np.sqrt(s2))
    dc = cmax / 5000
    c1 = h / (y * (p - k)) * 100
    c2 = (h + (f + F) / H) / ((p - k) * y) * 100
    c3 = h / ((p - k) - (f + F) / K) / y * 100

    cc = np.unique(np.concatenate((
        np.linspace(0.00001 * dc, cmax, 5000),
        [c1, c2, c3]
    )))

    # Reserve
    tc, qc, mc = reserve(moy, s2, cc, itype=1)

    v1 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) / M
    v2 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc / H
    v3 = (p - k) * tc * mc / 100 * y - tc * h - m - (f + F) * tc * mc / 100 * y / K

    m2 = 1.1 * np.max([v1, v2, v3])
    m1 = np.min([v1, v2, v3])
    dy = 0.035 * (m2 - m1)

    # Calcul de la valeur minimale entre v1, v2 et v3 pour chaque teneur
    v_min = np.minimum(np.minimum(v1, v2), v3)

    # Recherche du profit optimal : max de cette valeur minimale
    idx_opt = np.argmax(v_min)
    c_opt = cc[idx_opt]
    profit_opt = v_min[idx_opt]

    # Calculer les teneurs d'équilibre
    ceq = []
    veq = []
    ind = []

    def check_equilibre(vA, vB):
        t_val, i_val = np.min(np.abs(vA - vB)), np.argmin(np.abs(vA - vB))
        seuil = 10 * np.max(np.abs(v1[:-1] - v1[1:]))
        if t_val < seuil:
            ceq.append(cc[i_val])
            veq.append(vA[i_val])
            ind.append(i_val)

    check_equilibre(v1, v2)
    check_equilibre(v1, v3)
    check_equilibre(v2, v3)

    # Liste des teneurs limites
    limites = [c1, c2, c3]

    # Liste complète des teneurs limites + d'équilibre détectées
    t = np.array(limites + ceq)

    # Recherche de la teneur la plus proche de c_opt dans cette liste
    i = np.argmin(np.abs(c_opt - t))

    # Repositionnement de la teneur optimale sur la grille exacte
    c_opt = t[i]

    # Attribution des labels
    labels_limites = ['Mine Limite', 'Concentrateur Limite', 'Marché Limite']
    labels_equilibre_possibles = [
        'Équilibre Mine-Concentrateur',
        'Équilibre Mine-Marché',
        'Équilibre Concentrateur-Marché'
    ]
    labels_equilibre = labels_equilibre_possibles[:len(ceq)]

    labels = labels_limites + labels_equilibre
    nature = labels[i]

    # Fixe la fenêtre graphique selon ta demande (centrée x entre 0.5 et 1, y entre 0 et 2)
    xmin, xmax = 0.5, 1
    ymin, ymax = 0, 1.75

    plt.figure(figsize=(10,6))
    plt.plot(cc, v1, '-', label='Mine')
    plt.plot(cc, v2, '--', label='Concentrateur')
    plt.plot(cc, v3, ':', label='Marché')
    plt.grid(True)

    # Zone hachurée entre -100 et le minimum des trois courbes
    min_curve = np.minimum(np.minimum(v1, v2), v3)
    plt.fill_between(cc, -100, min_curve, color='grey', alpha=0.3, hatch='///')

    plt.title(f'y={y:.2f}; p={p}; k={k}; h={h}; m={m}; f={f}; F={F}; M={M}; H={H}; K={K}; Moyenne={moy}, Variance={s2}', fontsize=10)
    plt.xlabel('Teneur de coupure (%)', fontsize=12)
    plt.ylabel('Profit / t. minéralisée ($)', fontsize=12)

    dy_text = 0.035 * (m2 - m1)

    for v, label in zip([v1, v2, v3], ['c1', 'c2', 'c3']):
        idx = np.argmax(v)
        if xmin <= cc[idx] <= xmax and ymin <= v[idx] <= ymax:
            plt.text(cc[idx], v[idx] + dy_text, label, fontsize=10, fontweight='bold', color='black')
            plt.vlines(cc[idx], v[idx], v[idx] + 0.8 * dy_text, colors='black', linewidth=1.5)

    pairs = [(v1, v2, 'c12'), (v1, v3, 'c13'), (v2, v3, 'c23')]
    for vA, vB, label in pairs:
        diff = np.abs(vA - vB)
        idx = np.argmin(diff)
        seuil = 0.1 * (np.max([vA.max(), vB.max()]) - np.min([vA.min(), vB.min()]))
        if diff[idx] < seuil:
            if xmin <= cc[idx] <= xmax and ymin <= vA[idx] <= ymax:
                plt.text(cc[idx], vA[idx] + dy_text, label, fontsize=10, fontweight='bold', color='red')
                plt.vlines(cc[idx], vA[idx], vA[idx] + 0.8 * dy_text, colors='red', linewidth=1.5)

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.legend(fontsize=10)
    plt.show()

    # Créer le tableau des résultats
    # Pour éviter une erreur si moins de 3 équilibres détectés, on complète avec None
    ceq_complete = ceq + [None]*(3 - len(ceq))
    df_data = [
        ["Optimale",        f"{c_opt:.2f} %",        f"{profit_opt:.2f} $",     nature],
        ["Limite C1",       f"{c1:.2f} %",           f"{v1[np.argmax(v1)]:.2f} $", "Mine"],
        ["Limite C2",       f"{c2:.2f} %",           f"{v2[np.argmax(v2)]:.2f} $", "Concentrateur"],
        ["Limite C3",       f"{c3:.2f} %",           f"{v3[np.argmax(v3)]:.2f} $", "Marché"]
    ]

    # Ajouter les équilibres s'ils existent
    labels_equilibre_short = ["Équilibre C1-C2", "Équilibre C1-C3", "Équilibre C2-C3"]
    for i_eq, c_val in enumerate(ceq_complete):
        if c_val is not None:
            if i_eq == 0:
                idx_eq = np.argmin(np.abs(v1 - v2))
                profit_eq = v1[idx_eq]
            elif i_eq == 1:
                idx_eq = np.argmin(np.abs(v1 - v3))
                profit_eq = v1[idx_eq]
            elif i_eq == 2:
                idx_eq = np.argmin(np.abs(v2 - v3))
                profit_eq = v2[idx_eq]
            df_data.append([labels_equilibre_short[i_eq], f"{c_val:.2f} %", f"{profit_eq:.2f} $", labels_equilibre[i_eq]])

    df = pd.DataFrame(df_data, columns=["Type", "Teneur (%)", "Profit", "Nature"])

    display(df)

# Widgets pour m, k et p
style = {'description_width': 'initial'}
layout = widgets.Layout(width='300px')

w_s2 = widgets.BoundedFloatText(value=4, min=1, max=8, step=0.25, description='s2', layout=layout, style=style)

def update(s2):
    lane(s2=s2,)

widgets.interact(update, s2=w_s2)

interactive(children=(BoundedFloatText(value=4.0, description='s2', layout=Layout(width='300px'), max=8.0, min…

<function __main__.update(s2)>